In [51]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import pydicom
import numpy as np
from tqdm import tqdm
import torch
import cv2

In [3]:
# join first two tables
train_series=pd.read_csv('./test_series_descriptions.csv')
train=pd.read_csv('./train.csv')
train_label=pd.read_csv('./train_label_coordinates.csv')

In [4]:
df_train_step_1 = pd.merge(left=train, right=train_label, how='left', on='study_id').reset_index(drop=True)
df_train_step_1.head()

,study_id,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,...,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1,series_id,instance_number,condition,level,x,y
0,4003253,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,702807833.0,8.0,Spinal Canal Stenosis,L1/L2,322.831858,227.964602
1,4003253,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,702807833.0,8.0,Spinal Canal Stenosis,L2/L3,320.571429,295.714286
2,4003253,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,702807833.0,8.0,Spinal Canal Stenosis,L3/L4,323.030303,371.818182
3,4003253,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,702807833.0,8.0,Spinal Canal Stenosis,L4/L5,335.292035,427.327434
4,4003253,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,702807833.0,8.0,Spinal Canal Stenosis,L5/S1,353.415929,483.964602


In [12]:
# join with third table
df_train = pd.merge(left=df_train_step_1, right=train_series, how='left', on=['study_id', 'series_id']).reset_index(drop=True)
df_train.head()

part_1 = os.listdir('./train_images')
part_1 = list(filter(lambda x: x.find('.DS') == -1, part_1))
p1 = [(x, f"./train_images/{x}") for x in part_1]
meta_obj = { p[0]: { 'folder_path': p[1], 
                    'SeriesInstanceUIDs': [] 
                   } 
            for p in p1 }
for m in meta_obj:
    meta_obj[m]['SeriesInstanceUIDs'] = list(
        filter(lambda x: x.find('.DS') == -1, 
               os.listdir(meta_obj[m]['folder_path'])
              )
    )

In [13]:
for m in meta_obj:
    meta_obj[m]['SeriesInstanceUIDs'] = list(
        filter(lambda x: x.find('.DS') == -1, 
               os.listdir(meta_obj[m]['folder_path'])
              )
    )

In [15]:
# grabs the correspoding series descriptions
for k in tqdm(meta_obj):
    for s in meta_obj[k]['SeriesInstanceUIDs']:
        if 'SeriesDescriptions' not in meta_obj[k]:
            meta_obj[k]['SeriesDescriptions'] = []
        try:
            meta_obj[k]['SeriesDescriptions'].append(
                df_train[(df_train['study_id'] == int(k)) & 
                (df_train['series_id'] == int(s))]['series_description'].iloc[0])
        except:
            print("Failed on", s, k)

 62%|██████▏   | 1227/1975 [00:00<00:00, 1533.14it/s]

Failed on 3636216534 3008676218
Failed on 542282425 3008676218


 78%|███████▊  | 1538/1975 [00:01<00:00, 1516.61it/s]

Failed on 3892989905 3637444890


100%|██████████| 1975/1975 [00:01<00:00, 1522.75it/s]


In [16]:
df_train.columns

Index(['study_id', 'spinal_canal_stenosis_l1_l2',
       'spinal_canal_stenosis_l2_l3', 'spinal_canal_stenosis_l3_l4',
       'spinal_canal_stenosis_l4_l5', 'spinal_canal_stenosis_l5_s1',
       'left_neural_foraminal_narrowing_l1_l2',
       'left_neural_foraminal_narrowing_l2_l3',
       'left_neural_foraminal_narrowing_l3_l4',
       'left_neural_foraminal_narrowing_l4_l5',
       'left_neural_foraminal_narrowing_l5_s1',
       'right_neural_foraminal_narrowing_l1_l2',
       'right_neural_foraminal_narrowing_l2_l3',
       'right_neural_foraminal_narrowing_l3_l4',
       'right_neural_foraminal_narrowing_l4_l5',
       'right_neural_foraminal_narrowing_l5_s1',
       'left_subarticular_stenosis_l1_l2', 'left_subarticular_stenosis_l2_l3',
       'left_subarticular_stenosis_l3_l4', 'left_subarticular_stenosis_l4_l5',
       'left_subarticular_stenosis_l5_s1', 'right_subarticular_stenosis_l1_l2',
       'right_subarticular_stenosis_l2_l3',
       'right_subarticular_stenosis_l3_l4',
 

In [17]:
# dicom
import pydicom as dicom
mapping = {
    'normal/mild': 1,
    'moderate': 2,
    'severe': 3
}
def standardize_and_map(column, mapping):
    return column.str.lower().str.strip().map(mapping)

# List of columns to apply the transformation
columns_to_transform = ['spinal_canal_stenosis_l1_l2',
       'spinal_canal_stenosis_l2_l3', 'spinal_canal_stenosis_l3_l4',
       'spinal_canal_stenosis_l4_l5', 'spinal_canal_stenosis_l5_s1',
       'left_neural_foraminal_narrowing_l1_l2',
       'left_neural_foraminal_narrowing_l2_l3',
       'left_neural_foraminal_narrowing_l3_l4',
       'left_neural_foraminal_narrowing_l4_l5',
       'left_neural_foraminal_narrowing_l5_s1',
       'right_neural_foraminal_narrowing_l1_l2',
       'right_neural_foraminal_narrowing_l2_l3',
       'right_neural_foraminal_narrowing_l3_l4',
       'right_neural_foraminal_narrowing_l4_l5',
       'right_neural_foraminal_narrowing_l5_s1',
       'left_subarticular_stenosis_l1_l2', 'left_subarticular_stenosis_l2_l3',
       'left_subarticular_stenosis_l3_l4', 'left_subarticular_stenosis_l4_l5',
       'left_subarticular_stenosis_l5_s1', 'right_subarticular_stenosis_l1_l2',
       'right_subarticular_stenosis_l2_l3',
       'right_subarticular_stenosis_l3_l4',
       'right_subarticular_stenosis_l4_l5',
       'right_subarticular_stenosis_l5_s1']

In [18]:
# Apply the function to the specified columns
for column in columns_to_transform:
    df_train[column] = standardize_and_map(df_train[column], mapping)

In [71]:
test_df = df_train[df_train['study_id'] == 4003253]
test_df

,study_id,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,...,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1,series_id,instance_number,condition,level,x,y,series_description
0,4003253,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,...,1.0,1.0,1.0,7.028078e+08,8.0,Spinal Canal Stenosis,L1/L2,322.831858,227.964602,NaN
1,4003253,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,...,1.0,1.0,1.0,7.028078e+08,8.0,Spinal Canal Stenosis,L2/L3,320.571429,295.714286,NaN
2,4003253,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,...,1.0,1.0,1.0,7.028078e+08,8.0,Spinal Canal Stenosis,L3/L4,323.030303,371.818182,NaN
3,4003253,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,...,1.0,1.0,1.0,7.028078e+08,8.0,Spinal Canal Stenosis,L4/L5,335.292035,427.327434,NaN
4,4003253,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,...,1.0,1.0,1.0,7.028078e+08,8.0,Spinal Canal Stenosis,L5/S1,353.415929,483.964602,NaN
5,4003253,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,...,1.0,1.0,1.0,1.054714e+09,4.0,Right Neural Foraminal Narrowing,L4/L5,187.961759,251.839388,NaN
6,4003253,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,...,1.0,1.0,1.0,1.054714e+09,4.0,Right Neural Foraminal Narrowing,L5/S1,198.240918,285.613767,NaN
7,4003253,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,...,1.0,1.0,1.0,1.054714e+09,5.0,Right Neural Foraminal Narrowing,L3/L4,187.227533,210.722753,NaN
8,4003253,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,...,1.0,1.0,1.0,1.054714e+09,6.0,Right Neural Foraminal Narrowing,L1/L2,194.569790,127.755258,NaN
9,4003253,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,...,1.0,1.0,1.0,1.054714e+09,6.0,Right Neural Foraminal Narrowing,L2/L3,191.632887,165.934990,NaN


In [ ]:
# 1 image (x .dcm files) = ....
# 1 patient = 3 analyses

In [ ]:
def resize_image(part_1, study_id, series_id, target_size=(224, 224)):
    dicom = pydicom.dcmread(os.path.join(part_1, str(study_id), str(series_id)))

    # Convert the DCM image to a numpy array
    image_array = dicom.pixel_array
    # Resize the image using OpenCV
    resized_image = cv2.resize(image_array, target_size)
    # Store the resized image in a numpy array
    resized_image_array = np.array(resized_image)

    return resized_image_array

In [68]:
image_array = []

for study_id in part_1:
    study_id_dir = os.listdir(f'./train_images/{study_id}')
    for series_id in study_id_dir:
        series_id_dir = os.listdir(f'./train_images/{study_id}/{series_id}')
        for instance in series_id_dir:
            resized_image_array = resize_image()
            image_array.append(resized_image_array)



# study_id_dir = os.listdir(f'./train_images/100206310')
# study_id_dir

1.dcm
10.dcm
11.dcm
12.dcm
13.dcm
14.dcm
15.dcm
16.dcm
17.dcm
18.dcm
19.dcm
2.dcm
20.dcm
21.dcm
22.dcm
23.dcm
24.dcm
25.dcm
26.dcm
27.dcm
28.dcm
29.dcm
3.dcm
30.dcm
31.dcm
32.dcm
33.dcm
34.dcm
35.dcm
36.dcm
37.dcm
38.dcm
39.dcm
4.dcm
40.dcm
41.dcm
42.dcm
43.dcm
44.dcm
45.dcm
46.dcm
47.dcm
48.dcm
49.dcm
5.dcm
50.dcm
51.dcm
52.dcm
53.dcm
54.dcm
55.dcm
56.dcm
57.dcm
58.dcm
59.dcm
6.dcm
60.dcm
7.dcm
8.dcm
9.dcm
1.dcm
10.dcm
11.dcm
12.dcm
13.dcm
14.dcm
15.dcm
16.dcm
17.dcm
18.dcm
2.dcm
3.dcm
4.dcm
5.dcm
6.dcm
7.dcm
8.dcm
9.dcm
1.dcm
10.dcm
11.dcm
12.dcm
13.dcm
14.dcm
15.dcm
16.dcm
17.dcm
18.dcm
2.dcm
3.dcm
4.dcm
5.dcm
6.dcm
7.dcm
8.dcm
9.dcm
1.dcm
10.dcm
11.dcm
12.dcm
13.dcm
14.dcm
15.dcm
16.dcm
17.dcm
18.dcm
19.dcm
2.dcm
20.dcm
21.dcm
22.dcm
23.dcm
24.dcm
25.dcm
26.dcm
27.dcm
28.dcm
29.dcm
3.dcm
30.dcm
31.dcm
32.dcm
33.dcm
34.dcm
35.dcm
36.dcm
37.dcm
38.dcm
39.dcm
4.dcm
40.dcm
41.dcm
42.dcm
5.dcm
6.dcm
7.dcm
8.dcm
9.dcm
1.dcm
10.dcm
11.dcm
12.dcm
13.dcm
14.dcm
15.dcm
16.dcm
17.dcm
18.dcm
2

In [70]:
# meta_obj = {p[0]: { 'study_id': study_id, 
#                     'SeriesID': series_id,
#                     'resizes_img': ...
#                    }}


p1 = [(x, f"./train_images/{x}") for x in part_1]
meta_obj = { p[0]: { 'folder_path': p[1], 
                    'SeriesInstanceUIDs': [] 
                   } for p in p1 }

p1

[('100206310', './train_images/100206310'),
 ('1002894806', './train_images/1002894806'),
 ('1004726367', './train_images/1004726367'),
 ('1008446160', './train_images/1008446160'),
 ('1009445512', './train_images/1009445512'),
 ('1009905322', './train_images/1009905322'),
 ('1012375618', './train_images/1012375618'),
 ('1013589491', './train_images/1013589491'),
 ('1013791258', './train_images/1013791258'),
 ('1018005303', './train_images/1018005303'),
 ('1019430579', './train_images/1019430579'),
 ('1020394063', './train_images/1020394063'),
 ('1025265129', './train_images/1025265129'),
 ('1028684462', './train_images/1028684462'),
 ('1028909382', './train_images/1028909382'),
 ('1035170868', './train_images/1035170868'),
 ('1036203708', './train_images/1036203708'),
 ('1038453736', './train_images/1038453736'),
 ('1039182563', './train_images/1039182563'),
 ('1040921274', './train_images/1040921274'),
 ('1047914296', './train_images/1047914296'),
 ('1050200728', './train_images/1050

In [ ]:
images = []
labels = []
for index, row in df.iterrows():
    image_folder = f"train/{row['study_id']}"
    image_paths = [os.path.join(image_folder, image_name) for image_name in os.listdir(image_folder) if image_name.endswith('.dcm')]
    for image_path in image_paths:
        images.append(preprocess_image(image_path))
        label = [row[f'deformity{i}'] for i in range(1, len(df.columns))]
        labels.append(label)